# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
# Import cities file as DataFrame
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Print as a check
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ahipara,-35.1667,173.1667,68.86,84,97,15.97,NZ,1613425013
1,korhogo,9.4500,-5.6333,86.40,41,95,4.38,CI,1613425013
2,taltal,-25.4000,-70.4833,68.47,67,21,4.43,CL,1613425013
3,rudraprayag,30.2833,78.9833,52.92,49,6,4.16,IN,1613425014
4,dingle,10.9995,122.6711,74.05,89,98,6.98,PH,1613425014


In [25]:
# Check dataframe types
cities_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = cities_df["Humidity"].astype(float)

In [27]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
# Use .loc to filter on weather criteria and call dataframe hotel_df

hotel_df = cities_df.loc[((cities_df['Max Temp'] > 70) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness'] == 0))]

# Print df for visual check
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,poum,-20.2333,164.0167,80.83,77,0,9.33,NC,1613425027
97,avarua,-21.2078,-159.7750,84.20,74,0,6.91,CK,1613425035
110,saint-philippe,-21.3585,55.7679,73.45,69,0,5.75,RE,1613425038
123,ugoofaaru,5.6667,73.0000,80.65,65,0,4.74,MV,1613425041
187,tahoua,14.8888,5.2692,84.20,9,0,4.61,NE,1613425055
192,saint-joseph,-21.3667,55.6167,73.56,69,0,5.75,RE,1613425056
281,palmares do sul,-30.2578,-50.5097,77.25,72,0,5.70,BR,1613425076
315,xochihuehuetlan,17.9167,-98.4333,87.57,11,0,5.39,MX,1613425084
344,perth,-31.9333,115.8333,70.86,83,0,6.91,AU,1613425090
371,yarada,17.6500,83.2667,71.60,94,0,2.30,IN,1613425095


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
# add column called Hotel Name
hotel_df.insert(0,'Hotel Name','')
hotel_df.head()

,Hotel Name,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,,poum,-20.2333,164.0167,80.83,77,0,9.33,NC,1613425027
97,,avarua,-21.2078,-159.7750,84.20,74,0,6.91,CK,1613425035
110,,saint-philippe,-21.3585,55.7679,73.45,69,0,5.75,RE,1613425038
123,,ugoofaaru,5.6667,73.0000,80.65,65,0,4.74,MV,1613425041
187,,tahoua,14.8888,5.2692,84.20,9,0,4.61,NE,1613425055


In [39]:
# Set params to search for hotels within 5000 meter radious 
params = {
    "radius": 50000,
    "types": "Lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # Get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request
    name_address = requests.get(base_url, params=params)
    
    # print(name_address.url)

    # Convert to json
    name_address = name_address.json()
    
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing a try-except is used to skip any that are missing data.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


C:\Users\pfenoughty\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable

markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))